<a href="https://colab.research.google.com/github/cocoaaa/Funs/blob/master/Copy_of_ICRA24_tutorial_geometry_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Riemann and Gauss meet Asimov: 2nd Tutorial on Geometric Methods in Robot Learning, Optimization and Control** #
# Riemannian optimization #
---

This Notebook aims at giving you a first glimpse of Riemannian optimization methods and of some of the existing Riemannian optimization libraries.

In this notebook, we will see:
*   How to implement a simple Riemannian gradient descent to optimize a controller gain on the SPD manifold,
*   How to use some of the more complex Riemannian optimization methods from Pymanopt,
*   How to use stochastic Riemannian optimizations approaches from Geoopt.

The parts of the code that you should implement are indicated with the comment **# TODO** throughout the notebook.





# **1. Optimization of an impedance controller gain with Riemannian gradient descent**
---
In this first part, our goal is to learn the stiffness matrix of an impedance controller. We represent the movement of a planar robot’s end-effector as
a unit point mass moving in the plane $\mathbb{R}^2$. The behavior of the end-effector is modelled as a spring-damper system: Given a desired position $\hat{\mathbf{x}}$, the end-effector acceleration is driven by the impedance controller $\ddot{\mathbf{x}} = \mathbf{K}^p (\hat{\mathbf{x}} - \mathbf{x}) - \mathbf{K}^d \dot{\mathbf{x}} $, where $\mathbf{K}^p$ and $\mathbf{K}^d \in\mathcal{S}_{++}^D$ are the stiffness and damping gains, respectively.

Given an demonstrated trajectory $\{ \mathbf{x}_t, \dot{\mathbf{x}}_t, \ddot{\mathbf{x}}_t \}_{t=0}^T$, our goal is to infer the stiffness gain of our controller, such that the robot can reproduce the demonstrated trajectory.


We start by installing and importing the required packages that we will use in the different examples.

In [ ]:
!pip install numpy
!pip install scipy
!pip install torch
!pip install torchdiffeq
!pip install jax
!pip install pymanopt==2.0.1
!pip install geoopt
!pip install matplotlib

In [ ]:
from typing import Union, Tuple
import numpy as np
import scipy.linalg
import torch
from torchdiffeq import odeint
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.path as mpath
import matplotlib.patches as mpatches

Define and display the demonstrated trajectory.

In [ ]:
# Demonstrated trajectory
# Position
ref_traj_x_np = np.array([[-5.41421356, -3.41421356],
 [-5.26257245, -0.68248427],
 [-4.85836787,  1.31069968],
 [-4.37086438,  2.28727136],
 [-3.89674053,  2.7051812 ],
 [-3.47967017,  2.87399588],
 [-3.1331609,   2.94139824],
 [-2.85560442,  2.9692274 ],
 [-2.63883762,  2.98172984],
 [-2.47266588,  2.98810324]])
ref_traj_x = torch.from_numpy(ref_traj_x_np)
nb_points = ref_traj_x_np.shape[0]

# Velocity
ref_traj_dx_np = np.array([[0. ,        0.        ],
 [0.58141116, 3.70644107],
 [0.84598923, 2.30142563],
 [0.88546181, 1.07768499],
 [0.80934858, 0.45388672],
 [0.68813533, 0.1836573 ],
 [0.55969636, 0.07486522],
 [0.44187708, 0.03232233],
 [0.34149055, 0.01553348],
 [0.25969998, 0.00853142]])
ref_traj_dx = torch.from_numpy(ref_traj_dx_np)

# Acceleration
ref_traj_ddx_np = np.array([[ 1.06568542e+00,  3.17296465e+01],
 [ 6.93707681e-01,  1.55189434e+00],
 [ 2.21675313e-01, -2.07292518e+00],
 [-6.33656592e-02, -1.43224769e+00],
 [-1.93525649e-01, -6.90170190e-01],
 [-2.32325929e-01, -2.92413406e-01],
 [-2.24805706e-01, -1.17028632e-01],
 [-1.97264018e-01, -4.61923722e-02],
 [-1.63734582e-01, -1.87786900e-02],
 [-1.31123463e-01, -8.27801519e-03]])
ref_traj_ddx = torch.from_numpy(ref_traj_ddx_np)

# Target
x_target_np = ref_traj_x_np[-1]
x_target = ref_traj_x[-1]

# Initial conditions
x0 = ref_traj_x[0]
dx0 = ref_traj_dx[0]

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.gca()
plt.plot(ref_traj_x_np[:, 0], ref_traj_x_np[:, 1], color='seagreen', linewidth=2)
for n in range(nb_points):
    plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='.', markersize=10, color='seagreen')
plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='*', markersize=15, color='seagreen')
ax.axis('equal')
plt.show()

We define an impedance controller

$\ddot{\mathbf{x}} = \mathbf{K}^p (\hat{\mathbf{x}} - \mathbf{x}) - \mathbf{K}^d \dot{\mathbf{x}}, $

for our robot to reproduce the demonstrated trajectory.
The stiffness gain $\mathbf{K}^p$ is given as a parameter, and the damping gain $\mathbf{K}^d$ is set so that the system is critically damped, i.e., $\mathbf{K}^d = 2 \sqrt{\mathbf{K}^p}$.

The resulting trajectory is then obtained by numerically integrating the acceleration $\ddot{\mathbf{x}}$ for $T$ timesteps.

In [ ]:
def point_mass_impedance_controller(x, dx, x_target, kp):
    # Compute sqrtm of Kp
    eigenvals, eigenvecs = torch.linalg.eigh(kp)
    sqrtm_kp = torch.mm(torch.mm(eigenvecs, torch.diag(torch.sqrt(eigenvals))), eigenvecs.T)
    # Compute the damping gain, such that the system is critically damped
    kd = 2. * sqrtm_kp

    # Acceleration = force / mass, with mass = 1.
    acceleration = (torch.mm(kp, (x_target - x)[:, None]) - torch.mm(kd, dx[:, None]))[:, 0]
    return acceleration

def impedance_controller_ode(t, xdx, x_target, kp):
    nb_dofs = int(xdx.shape[0] / 2.)
    x = xdx[:nb_dofs]
    dx = xdx[nb_dofs:]

    dxddx = torch.zeros_like(xdx)
    dxddx[:nb_dofs] = dx
    dxddx[nb_dofs:] = point_mass_impedance_controller(x, dx, x_target, kp)

    return dxddx

def integrated_trajectory_ivp(x0, dx0, t, x_target, kp):
    nb_dofs = x0.shape[0]
    xdx0 = torch.hstack((x0, dx0))
    res = odeint(lambda t, xdx: impedance_controller_ode(t, xdx, x_target, kp), xdx0, t, method='rk4')

    return res[..., :nb_dofs], res[..., nb_dofs:]

To visualize the influence of the stiffness gain on the reproduction, we draw some trajectories with different stiffness gains.

Change the stiffness gain in the cell below and observe the resulting trajectory (blue) compared to the demonstration (green). Remember that the stiffness gain must be a symmetric positive definite 2x2 matrix.

In [ ]:
# Define the stiffness gain
kp = np.array([[2.0, 0.], [0., .5]])  # TODO CHANGE

# Reproduce trajectory
t = torch.linspace(0., 5.0, nb_points)
repro_x, _ = integrated_trajectory_ivp(x0, dx0, t, x_target, torch.from_numpy(kp))
repro_x_np = repro_x.detach().numpy()

# Plot reference and resulting trajectory with the given gain
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
plt.plot(ref_traj_x_np[:, 0], ref_traj_x_np[:, 1], color='seagreen', linewidth=2)
plt.plot(repro_x_np[:, 0], repro_x_np[:, 1], color='navy', linewidth=2)
for n in range(nb_points):
    plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='.', markersize=10, color='seagreen')
    plt.plot(repro_x_np[n, 0], repro_x_np[n, 1], marker='.', markersize=10, color='navy')
plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='*', markersize=15, color='seagreen')
ax.axis('equal')
plt.show()


We now optimize the stiffness gain. To do so, we define our cost function as the norm of the residuals as in [1],

$f(\mathbf{K}^p) = \| \ddot{\mathbf{x}}_t - \mathbf{K}^p (\hat{\mathbf{x}} - \mathbf{x}_t) + \mathbf{K}^d \dot{\mathbf{x}}_t \|$,

where $\{\mathbf{x}_t, \dot{\mathbf{x}}_t, \ddot{\mathbf{x}}_t\}$ are the demonstrated position, velocity, and acceleration, respectively.

We define the stiffness gain estimation as the Riemannian optimization problem,

$\min_{\mathbf{K}^p\in\mathcal{S}^2_{++}} f(\mathbf{K}^p) $.

\\

[1] L. Rozo, S. Calinon, D. G. Caldwell, P. Jimnez, and C. Torras. **Learning Physical Collaborative Robot Behaviors From Human Demonstrations**, IEEE Trans. on Robotics 32, pp. 513–527, 2015.

We first define the cost function $f(\mathbf{K}^p)$ and the Euclidean gradient $\text{grad}_{\mathbb{R}} f(\mathbf{K}^p)$.

In [ ]:
def cost(kp):
  return np.linalg.norm(kp @ (x_target_np[:, None] - ref_traj_x_np.T) - ref_traj_ddx_np.T + kd @ ref_traj_dx_np.T)

def euclidean_gradient(kp):
  c = cost(kp)
  return (kp @ (x_target_np[:, None] - ref_traj_x_np.T) - ref_traj_ddx_np.T + kd @ ref_traj_dx_np.T) @ (x_target_np[None] - ref_traj_x_np) / c

We then define utility functions for the SPD manifold:
- The exponential map;
- The Riemannian gradient, computed from the Euclidean gradient.

In [ ]:
def spd_expmap(point, tangent_vector):
  p_inv_tv = np.linalg.solve(point, tangent_vector)
  return point @ scipy.linalg.expm(p_inv_tv)

def spd_euclidean_to_riemannian_gradient(point, euclidean_gradient):
  return 0.5 * point @ (euclidean_gradient + euclidean_gradient.T) @ point

We now solve the optimization $\min_{\mathbf{K}^p\in\mathcal{S}^2_{++}} f(\mathbf{K}^p) $ via Riemannian gradient. Implement the missing parts of the Riemannian gradient descent in the cell below. Then, observe the evolution of the cost, and gradient norm, as well as the resuling optimized stiffness.

In [ ]:
# Parameters
K = 30  # Number of iterations
step_size = 0.05
kd = np.array([[1.0, 0.0], [0.0, 1.0]])  # Constant estimate of the damping gain

# Initialization
kp_k = np.array([[1.0, 0.0], [0.0, 1.0]])

# Riemannian gradient descent
print('Iteration\t', 'Cost\t\t\t', 'Norm gradient')
print('---------\t', '------------------\t', '------------------')
for k in range(K):
  # Compute the Riemannian gradient
  g_k = ...  # TODO implement
  # Update step
  kp_k = ... # TODO implement

  # Compute current cost
  c_k = cost(kp_k)
  print(k, '\t\t', c_k, '\t', np.linalg.norm(g_k))

# Optimized gain
kp_opt = kp_k
print("\n\n Optimal Kp:\n", kp_opt)

We reproduce the trajectory with the optimal stiffness gain.

In [ ]:
# Reproduce trajectory
repro_x, _ = integrated_trajectory_ivp(x0, dx0, t, x_target, torch.from_numpy(kp_opt))
repro_x_np = repro_x.detach().numpy()

# Plot reference and resulting trajectory after optimizing the impedance gain
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
plt.plot(ref_traj_x_np[:, 0], ref_traj_x_np[:, 1], color='seagreen', linewidth=2)
plt.plot(repro_x_np[:, 0], repro_x_np[:, 1], color='navy', linewidth=2)
for n in range(nb_points):
    plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='.', markersize=10, color='seagreen')
    plt.plot(repro_x_np[n, 0], repro_x_np[n, 1], marker='.', markersize=10, color='navy')
plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='*', markersize=15, color='seagreen')
ax.axis('equal')
plt.show()

# **2. Optimization of an impedance controller gain with Pymanopt**
---
In this second part, we leverage some of the more complex Riemannian optimization methods implemented in the library Pymanopt to optimize our stiffness gain.

Pymanopt is used to optimize a cost function by executing the following steps:
- Instantiate the manifold on which our optimized variable lies;
- Define our cost function;
- Create a Pymanopt problem;
- Instantiate the Riemannian optimizer of our choice;
- Run the optimizer.

These different steps are implemented below and several solvers are imported. Take the time to try them out and to observe their evolution and the resuling optimized stiffness as dislayed after the cell.

In [ ]:
# Import pymanopt
import pymanopt
from pymanopt import Problem
from pymanopt.manifolds import SymmetricPositiveDefinite
from pymanopt.optimizers import ConjugateGradient, SteepestDescent, TrustRegions

In [ ]:
# Create SPD manifold of dimension 2 (2x2 SPD matrices)
manifold = SymmetricPositiveDefinite(2)

# Create pymanopt cost function
# The cost function must be decorated with a backend decorator. Here we use torch.
@pymanopt.function.pytorch(manifold)
def cost(kp):
  # Constant estimate of the damping gain
  kd = torch.from_numpy(np.array([[1.0, 0.0], [0.0, 1.0]]))
  # Cost
  return torch.norm(torch.mm(kp, (x_target - ref_traj_x).T) - ref_traj_ddx.T + torch.mm(kd, ref_traj_dx.T))

# Create pymanopt problem
problem = Problem(manifold=manifold, cost=cost)

# Instantiate optimizer
optimizer = ConjugateGradient()  # TODO CHANGE OPTIMIZER

# Optimize
result = optimizer.run(problem)
kp_opt = result.point
kp_opt_cost = result.cost

# Optimized gain
print("Optimal Kp:\n", kp_opt)

We reproduce the trajectory with the optimal stiffness gain.




In [ ]:
# Reproduce trajectory
repro_x, _ = integrated_trajectory_ivp(x0, dx0, t, x_target, torch.from_numpy(kp_opt))
repro_x_np = repro_x.detach().numpy()

# Plot reference and resulting trajectory after optimizing the impedance gain
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
plt.plot(ref_traj_x_np[:, 0], ref_traj_x_np[:, 1], color='seagreen', linewidth=2)
plt.plot(repro_x_np[:, 0], repro_x_np[:, 1], color='navy', linewidth=2)
for n in range(nb_points):
    plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='.', markersize=10, color='seagreen')
    plt.plot(repro_x_np[n, 0], repro_x_np[n, 1], marker='.', markersize=10, color='navy')
plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='*', markersize=15, color='seagreen')
ax.axis('equal')
plt.show()


# **3. Optimization of an impedance controller gain with Geoopt**
---
In this third part, we leverage stochastic Riemannian optimization methods implemented in the library Geoopt to optimize our stiffness gain.

Geoopt is used to optimize a cost function by executing the following steps:
- Define our cost function;
- Instantiate the manifold on which our optimized variable lies;
- Initialize the optimization variable as a Geoopt manifold parameter;
- Instantiate the Riemannian stochastic optimizer of our choice;
- Run the optimizer as a Pytorch optim.

These different steps are implemented below and several solvers are imported. Take the time to try them out and to observe their evolution and the resuling optimized stiffness as dislayed after the cell.

In [ ]:
from geoopt.manifolds import SymmetricPositiveDefinite
from geoopt import ManifoldParameter, ManifoldTensor
from geoopt.optim import RiemannianAdam, RiemannianSGD


In [ ]:
def cost(kp):
  # Constant estimate of the damping gain
  kd = torch.from_numpy(np.array([[1.0, 0.0], [0.0, 1.0]]))
  # Cost
  return torch.norm(torch.mm(kp, (x_target - ref_traj_x).T) - ref_traj_ddx.T + torch.mm(kd, ref_traj_dx.T))

In [ ]:
# Parameters
K = 100  # Number of iterations
lr = 1e-2

# Instantiate the manifold
manifold = SymmetricPositiveDefinite()

# Initialization of Kp as a Manifold parameter
kp = ManifoldParameter(torch.from_numpy(np.eye(2)), manifold=manifold)

# Instanciate the optimizer
optimizer = RiemannianAdam([kp], lr=1e-2)  # TODO CHANGE OPTIMIZER

# Riemannian gradient descent
print('Iteration\t', 'Cost\t\t\t')
print('---------\t', '------------------\t')
for k in range(K):
  loss = cost(kp)
  loss.backward()
  optimizer.step()

  print(k, '\t\t', loss.detach().numpy())

# Optimized gain
kp_opt = kp
print("\n\n Optimal Kp:\n", kp_opt.detach().numpy())

We reproduce the trajectory with the optimal stiffness gain.

In [ ]:
# Reproduce trajectory
repro_x, _ = integrated_trajectory_ivp(x0, dx0, t, x_target, kp_opt)
repro_x_np = repro_x.detach().numpy()

# Plot reference and resulting trajectory after optimizing the impedance gain
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
plt.plot(ref_traj_x_np[:, 0], ref_traj_x_np[:, 1], color='seagreen', linewidth=2)
plt.plot(repro_x_np[:, 0], repro_x_np[:, 1], color='navy', linewidth=2)
for n in range(nb_points):
    plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='.', markersize=10, color='seagreen')
    plt.plot(repro_x_np[n, 0], repro_x_np[n, 1], marker='.', markersize=10, color='navy')
plt.plot(ref_traj_x_np[n, 0], ref_traj_x_np[n, 1], marker='*', markersize=15, color='seagreen')
ax.axis('equal')
plt.show()